# Test out the pick em game before moving to its own .exe or .py

#### 2024-12-11

In [1]:
import json
import pandas as pd
import os
import random

## Load data

In [11]:
path_moves_per_pokemon = os.path.join("..", "..", "Scraping", "data", "moves_per_pokemon_orig_names.json")
path_pokemon_main_stats = os.path.join("..", "..", "Scraping", "data", "pokemon_main_stats_data.csv")
path_pokemon_moves_master = os.path.join("..", "..", "Scraping", "data", "pokemon_moves_data.csv")
path_type_effectiveness = os.path.join("..", "data", "type_effectiveness.json")

In [12]:
# Load moves per pokemon as dict
with open(path_moves_per_pokemon, 'r') as f:
    dict_moves_per_pokemon = json.load(f)

# Load type effectiveness as a dict
with open(path_type_effectiveness, 'r') as f:
    dict_type_effectiveness = json.load(f)

In [14]:
# Load dfs
## Each pokemon and their stats
df_pokemon_main_stats = pd.read_csv(path_pokemon_main_stats)

# Pokemon moves
df_pokemon_moves_master = pd.read_csv(path_pokemon_moves_master)
df_pokemon_moves_master['Accuracy'] = df_pokemon_moves_master['Accuracy'].replace('∞', '100').astype(float) # Replace '∞' with 100.0 and convert the column to floats


In [21]:
df_pokemon_moves_master

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
0,"10,000,000 Volt Thunderbolt",Electric,Special,195.0,NaN,1.0,Pikachu-exclusive Z-Move. High critical hit ra...,NaN
1,Absorb,Grass,Special,20.0,100.0,25.0,User recovers half the HP inflicted on opponent.,NaN
2,Accelerock,Rock,Physical,40.0,100.0,20.0,User attacks first.,NaN
3,Acid,Poison,Special,40.0,100.0,30.0,May lower opponent's Special Defense.,10.0
4,Acid Armor,Poison,Status,NaN,NaN,20.0,Sharply raises user's Defense.,NaN
...,...,...,...,...,...,...,...,...
929,Yawn,Normal,Status,NaN,NaN,10.0,Puts opponent to sleep in the next turn.,NaN
930,Zap Cannon,Electric,Special,120.0,50.0,5.0,Paralyzes opponent.,100.0
931,Zen Headbutt,Psychic,Physical,80.0,90.0,15.0,May cause flinching.,20.0
932,Zing Zap,Electric,Physical,80.0,100.0,10.0,May cause flinching.,30.0


## Random Selection

### Randomly select two pokemon without replacement and four of their moves without replacement

In [8]:
# Randomly select two Pokémon without replacement
selected_pokemon = random.sample(list(dict_moves_per_pokemon.keys()), 2)

In [9]:
# Select 4 random moves for each Pokémon
battle_pokemon = {
    pokemon: random.sample(dict_moves_per_pokemon[pokemon], 4)
    for pokemon in selected_pokemon
}

In [10]:
battle_pokemon

{'Sealeo': ['Roar', 'Bulldoze', 'Snore', 'Rock Tomb'],
 'Cryogonal': ['Protect', 'Haze', 'Self-Destruct', 'Icy Wind']}

## Pokemon battle simulation

In [22]:
import random

def pokemon_battle(battle_pokemon, df_pokemon_main_stats, df_pokemon_moves_master, dict_type_effectiveness):
    # Initialize Pokemon with their stats
    pokemon = {}
    for name, moves in battle_pokemon.items():
        stats = df_pokemon_main_stats[df_pokemon_main_stats['Name'] == name].iloc[0]
        pokemon[name] = {
            'hp': stats['HP'],
            'max_hp': stats['HP'],
            'attack': stats['Attack'],
            'defense': stats['Defense'],
            'sp_atk': stats['Sp. Atk'],
            'sp_def': stats['Sp. Def'],
            'speed': stats['Speed'],
            'type': stats['Type'],
            'moves': moves
        }
    
    # Determine order based on Speed
    order = sorted(pokemon.keys(), key=lambda x: pokemon[x]['speed'], reverse=True)
    
    # Battle loop
    while all(pokemon[p]['hp'] > 0 for p in pokemon):
        for attacker_name in order:
            if pokemon[attacker_name]['hp'] <= 0:
                continue
            
            defender_name = [p for p in pokemon if p != attacker_name][0]
            
            # Select a random move
            move = random.choice(pokemon[attacker_name]['moves'])
            move_data = df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == move].iloc[0]
            
            # Calculate damage
            power = move_data['Power'] if not pd.isna(move_data['Power']) else 0
            attacker = pokemon[attacker_name]
            defender = pokemon[defender_name]
            
            # Type effectiveness
            effectiveness = 1
            for att_type in attacker['type']:
                for def_type in defender['type']:
                    if att_type in dict_type_effectiveness and def_type in dict_type_effectiveness[att_type]:
                        effectiveness *= dict_type_effectiveness[att_type][def_type]
            
            # Damage calculation (simplified)
            if move_data['Category'] == 'Physical':
                damage = (((2 * 50 / 5 + 2) * power * attacker['attack'] / defender['defense']) / 50 + 2) * effectiveness
            elif move_data['Category'] == 'Special':
                damage = (((2 * 50 / 5 + 2) * power * attacker['sp_atk'] / defender['sp_def']) / 50 + 2) * effectiveness
            else:
                damage = 0
            
            damage = max(1, int(damage))  # Ensure at least 1 damage
            
            # Apply damage
            pokemon[defender_name]['hp'] = max(0, pokemon[defender_name]['hp'] - damage)
            
            print(f"{attacker_name} used {move} on {defender_name}!")
            print(f"It dealt {damage} damage. {defender_name}'s HP: {pokemon[defender_name]['hp']}/{pokemon[defender_name]['max_hp']}")
            
            if pokemon[defender_name]['hp'] <= 0:
                print(f"{defender_name} fainted! {attacker_name} wins!")
                return attacker_name
    
    return "Draw"  # This should never happen in a 1v1 battle, but just in case

In [23]:
pokemon_battle(battle_pokemon, df_pokemon_main_stats, df_pokemon_moves_master, dict_type_effectiveness)

Cryogonal used Self-Destruct on Sealeo!
It dealt 64 damage. Sealeo's HP: 26/90
Sealeo used Bulldoze on Cryogonal!
It dealt 33 damage. Cryogonal's HP: 47/80
Cryogonal used Icy Wind on Sealeo!
It dealt 34 damage. Sealeo's HP: 0/90
Sealeo fainted! Cryogonal wins!


'Cryogonal'

In [ ]:
## Now work on:
# Printing out whether super effective, not very effective
# PP
# Effects
# Simplified? damage calc - what are we missing?
"""
Note that this is a simplified version and doesn't include all Pokémon battle mechanics 
 (such as status effects, critical hits, or move PP). 
 You can expand on this base to add more complex mechanics as needed.
"""

In [25]:
df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == 'Self-Destruct']

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
703,Self-Destruct,Normal,Physical,200.0,100.0,5.0,User faints.,NaN
